In [60]:
import os
from docx.enum.section import WD_ORIENT
from docx.enum.table import WD_TABLE_ALIGNMENT
import pandas as pd
from typing import List
from docx.shared import Pt, Inches, Cm
from docx import Document
import pypandoc

import sys
sys.path.append('../src')
import cequalw2 as w2

In [61]:
def change_orientation(document, orientation: str):
    current_section = document.sections[-1]
    new_width, new_height = current_section.page_height, current_section.page_width
    if orientation.lower() == 'landscape':
        current_section.orientation = WD_ORIENT.LANDSCAPE
    elif orientation.lower() == 'portrait':
        current_section.orientation = WD_ORIENT.LANDSCAPE
    else:
        raise ValueError('Specify "landscape" or "portrait" for the orientation.')
    current_section.page_width = new_width
    current_section.page_height = new_height


def format_cell(cell, **kwargs):
    # Parse keyword arguments
    font_size = kwargs.get('font_size', None)
    bold = kwargs.get('bold', None)
    paragraph_font_size = kwargs.get('paragraph_font_size', 11)
    table_font_size = kwargs.get('table_font_size', 10)

    # Get pointer
    paragraph = cell.paragraphs[0]
    run = paragraph.runs[0]
    
    # Format the cell
    if font_size:
        run.font.size = Pt(table_font_size)
    if bold:
        run.bold = True


In [69]:
df1 = w2.read('data/BerlinMilton2006/2006_DeerCrk_Qin.npt', 2006, ['Inflow'])
df2 = w2.read('data/BerlinMilton2006/2006_DeerCrk_Cin.npt', 2006, ['TDS', 'SO4', 'Cl', 'ISS', 'OP', 'NH4', 'NOx', 'Fe', 'LDOM', 'RDOM', 'LPOM', 'RPOM', 'BG', 'DIAT', 'OTH', 'DO'])


In [70]:
df1.describe()

,Inflow
count,8761.000000
mean,1.553077
std,2.179551
min,0.000000
25%,0.350000
50%,0.800000
75%,1.740000
max,20.460000


In [71]:
df2.describe()

,TDS,SO4,Cl,ISS,OP,NH4,NOx,Fe,LDOM,RDOM,LPOM,RPOM,BG,DIAT,OTH,DO
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.0,11.0,11.000000
mean,231.545455,32.027273,35.645455,12.290909,0.013091,0.139727,0.480000,0.497636,0.545455,6.281818,1.363636,0.354545,0.363636,0.1,0.2,9.009091
std,27.045752,6.317451,5.133296,9.527797,0.008723,0.098228,0.230911,0.217895,0.216165,2.503924,0.555469,0.136848,0.233550,0.0,0.0,2.495778
min,188.000000,21.400000,29.200000,0.000000,0.005000,0.062000,0.170000,0.280000,0.300000,3.200000,0.700000,0.200000,0.000000,0.1,0.2,6.500000
25%,218.000000,28.800000,32.300000,5.300000,0.006000,0.078000,0.320000,0.343000,0.350000,4.300000,0.900000,0.250000,0.250000,0.1,0.2,7.400000
50%,230.000000,29.500000,35.200000,15.300000,0.012000,0.110000,0.410000,0.418000,0.500000,6.100000,1.300000,0.300000,0.500000,0.1,0.2,8.000000
75%,247.000000,37.750000,38.000000,15.900000,0.014500,0.123000,0.565000,0.606000,0.700000,8.000000,1.750000,0.450000,0.500000,0.1,0.2,9.750000
max,285.000000,41.500000,47.100000,32.200000,0.029000,0.334000,0.870000,0.868000,0.900000,10.900000,2.400000,0.600000,0.500000,0.1,0.2,13.900000


In [72]:
# Example data for the report
model_name = "CE-QUAL-W2"
location = "Berlin and Milton"
time_period = "2006"

# Generate the report
report = generate_report([df1, df2], model_name=model_name, location=location, time_period=time_period,
    table_font_size=6)

# Save the report to a file
report.save("water_quality_report.docx")

In [73]:
pypandoc.convert_file('water_quality_report.docx', 'pdf', outputfile='water_quality_report.pdf', extra_args=['--pdf-engine=xelatex', '-V', 'geometry:margin=1.5cm,landscape'])

''